In [11]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy.stats import linregress

In [12]:
PEARSONBOUND = 0.95

In [13]:
basicFileName="basic_data.csv"
basicDF=pd.read_csv('../data/'+basicFileName)
basicDF.drop_duplicates(inplace=True)
basicDF['Symbol'].to_csv('../data/'+'ETFs.csv',index=False)
#Inception记录股票上市时间
firstDF=basicDF[['Symbol','Fund Name','Assets','Category','Index','Inception','Volume']]
print(len(firstDF))
firstDF.dropna(inplace=True,subset=['Category'])
firstDF.dropna(inplace=True,subset=['Index'])
firstDF.dropna(inplace=True,subset=['Assets'])
print(len(firstDF))
firstDF = firstDF.loc[firstDF.groupby('Index')['Assets'].idxmax()]

# 重置索引（可选）
firstDF = firstDF.reset_index(drop=True)

print(len(firstDF))

firstDF[['Index','Symbol']].to_csv("./111.csv")


3963
2136
1823


/tmp/ipykernel_3001002/1385428522.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  firstDF.dropna(inplace=True,subset=['Category'])
/tmp/ipykernel_3001002/1385428522.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  firstDF.dropna(inplace=True,subset=['Index'])
/tmp/ipykernel_3001002/1385428522.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  firstDF.dropna(inplace=True,subset=['Assets'])


In [14]:
#计算每类股票的资产总和
def category_assets(df):
    require_colums= ["Category","Assets"]
    for col in require_colums:
        if col not in firstDF.columns:
            print("Error: Missing column "+col)
            return

    df["Assets"]=pd.to_numeric(df["Assets"],errors='coerce')
    categorys=df.groupby("Category")["Assets"].sum().reset_index()
    categorys=categorys.sort_values("Assets",ascending=False)
    
    print("\nNet Assets by Category:\n")
    
    
    categorys.to_csv("../data/category_assets.csv")
    return categorys

categorys=category_assets(firstDF)
categorys.to_csv("../data/categorys.csv")
print(categorys["Category"])
# print(categorys[categorys["Category"]=="Large Blend"])


Net Assets by Category:

49                         Large Blend
50                        Large Growth
51                         Large Value
25                 Foreign Large Blend
88                          Technology
                    ...               
19                       Equity Hedged
34           Global Large-Stock Growth
62    Moderately Aggressive Allocation
54                   Long-Short Equity
66        Muni California Intermediate
Name: Category, Length: 95, dtype: object


In [19]:
#Setp3:删除流动性和资产小于 val 的基金

def filter_fund(df,categorys):
    require_colums= ["Category","Assets","Volume"]
    for col in require_colums:
        if col not in firstDF.columns:
            print("Error: Missing column "+col)
            return

    totAssets=categorys["Assets"].sum()
    print("Total Assets: ",totAssets)
        
    cateVal={"-1":-1}
    for etf in df["Category"]:
        cateVal[etf]=categorys[categorys["Category"]==etf]["Assets"].values[0]
    
    df["calval"]=df["Category"].map(cateVal)
    
    #保留Assets大于0.1%总资产的基金和大于5%该类资产的基金
    filtered_df = df[
        (df['Assets'] > 0.001 * totAssets) |
        (df['Assets'] > df['Category'].map(lambda ca: 0.05 * cateVal.get(ca, 0)))
    ]
    
    filtered_df = df[~df['Category'].str.contains('Leverage', na=False)]
    filtered_df.to_csv("../data/filter_fund.csv")
    
    filtered_df["Symbol"].to_csv("../data/ETF1s.csv",index=False)
    return df

df2=filter_fund(firstDF,categorys)
categorys=category_assets(firstDF)


Total Assets:  7904078304875.0

Net Assets by Category:



In [16]:
# linear_regression(SPY,ETF)
# find alpha and beta and 皮尔逊相关系数
def linear_regression(basicDf1,basicDf2):
    basicDf1["Date"]=pd.to_datetime(basicDf1["date"]).dt.date
    basicDf2["Date"]=pd.to_datetime(basicDf2["date"]).dt.date
    common_dates=set(basicDf1['Date']).intersection(set(basicDf2['Date']))
    common_dates=sorted(list(common_dates))
    df1=basicDf1[basicDf1['Date'].isin(common_dates)]
    df2=basicDf2[basicDf2['Date'].isin(common_dates)]
    df=pd.merge(df1[['Date','close']],df2[['Date','close']],on='Date',suffixes=('_1','_2'))
    df['return1']=df1['close'].pct_change()
    df['return2']=df2['close'].pct_change()
    df=df.dropna()
    x = sm.add_constant(df['return1'])
    y = df['return2']
    
    model = sm.OLS(y, x).fit()
    alpha = model.params['const']
    beta = model.params['return1']
    
    correlation=np.corrcoef(df['return1'],df['return2'])[0,1]
    
    
    # print("Alpha:", alpha)
    # print("Beta:", beta)
    # # 绘制 SPY 和 ETF 收益率的散点图和回归线
    # plt.figure(figsize=(10, 6))
    # plt.scatter(df['return1'], df['return2'], label='Data Points', alpha=0.6)
    # plt.plot(df['return1'], alpha + beta * df['return1'], color='red', label='Regression Line')

    # # 添加图表标题和标签
    # plt.title('ETF Returns vs SPY Returns (Alpha and Beta)', fontsize=16)
    # plt.xlabel('SPY Returns', fontsize=12)
    # plt.ylabel('ETF Returns', fontsize=12)
    # plt.legend()
    # plt.grid(True)
    # plt.show()
    return alpha,beta,(abs(correlation)>PEARSONBOUND)
    

dfSPY=pd.read_csv("../data/daily_data/SPY_daily.csv")
dfVOO=pd.read_csv("../data/daily_data/QQQ_daily.csv")
a,b,c=linear_regression(dfSPY,dfVOO)
print(a)
print(b)
print(c)

0.0001664940081468543
1.1054567265769366
False
